In [13]:
import pygad
import numpy as np
import pandas as pd

In [14]:
df = pd.DataFrame([
    {"name": "Refrigerator A", "space": 0.751, "price": 999.90, "quantity": 2},
    {"name": "Cell phone", "space": 0.0000899, "price": 2911.12, "quantity": 5},
    {"name": "TV 55'", "space": 0.400, "price": 4346.99, "quantity": 2},
    {"name": "TV 50'", "space": 0.290, "price": 3999.90, "quantity": 2},
    {"name": "TV 42'", "space": 0.200, "price": 2999.00, "quantity": 3},
    {"name": "Notebook A", "space": 0.00350, "price": 2499.90, "quantity": 4},
    {"name": "Ventilator", "space": 0.496, "price": 199.90, "quantity": 2},
    {"name": "Microwave A", "space": 0.0424, "price": 308.66, "quantity": 3},
    {"name": "Microwave B", "space": 0.0544, "price": 429.90, "quantity": 3},
    {"name": "Microwave C", "space": 0.0319, "price": 299.29, "quantity": 3},
    {"name": "Refrigerator B", "space": 0.635, "price": 849.00, "quantity": 2},
    {"name": "Refrigerator C", "space": 0.870, "price": 1199.89, "quantity": 2},
    {"name": "Notebook B", "space": 0.498, "price": 1999.90, "quantity": 3},
    {"name": "Notebook C", "space": 0.527, "price": 3999.00, "quantity": 3}
])

df = df.loc[df.index.repeat(df['quantity'])].drop(columns=['quantity']).reset_index(drop=True)

In [15]:
# Масиви для генетичного алгоритму
spaces = df["space"].values
prices = df["price"].values
names = df["name"].values

# Обмеження на об'єм вантажівки
limit = 5

# Фітнес-функція
def fitness_function(ga_instance, solution, solution_idx):
    total_space = np.sum(solution * spaces)
    total_price = np.sum(solution * prices)
    return total_price if total_space <= limit else 0

# Генетичний алгоритм
ga = pygad.GA(
    num_generations=100,
    num_parents_mating=5,
    fitness_func=fitness_function,
    sol_per_pop=20,
    num_genes=len(df),
    gene_type=int,
    init_range_low=0,
    init_range_high=2,
    mutation_num_genes=2,
    parent_selection_type="tournament",
    keep_parents=2,
    crossover_type="single_point",
    mutation_type="random"
)

ga.run()

# Найкраще рішення
solution, solution_fitness, _ = ga.best_solution()
print(f"Найкраща вартість: {solution_fitness}")
print(f"Найкраще рішення: {solution}")
print("Обрані продукти:")

selected_products = df[solution == 1]
selected_products

Найкраща вартість: 39807.05
Найкраще рішення: [0 0 1 0 1 1 0 0 0 1 1 0 1 1 0 1 1 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 1
 0 1]
Обрані продукти:


,name,space,price
2,Cell phone,0.00009,2911.12
4,Cell phone,0.00009,2911.12
5,Cell phone,0.00009,2911.12
9,TV 50',0.29000,3999.90
10,TV 50',0.29000,3999.90
12,TV 42',0.20000,2999.00
13,TV 42',0.20000,2999.00
15,Notebook A,0.00350,2499.90
16,Notebook A,0.00350,2499.90
17,Notebook A,0.00350,2499.90


In [16]:
import plotly.express as px


figure = px.line(
    x=range(1, len(ga.best_solutions_fitness) + 1), 
    y=ga.best_solutions_fitness, 
    title='Зміна загальної ціни товарів у вантажівці за поколіннями',
    labels={'x': 'Покоління', 'y': 'Найкраща ціна'}
)
figure.show()